### Класс TimeSeriesFeatures()

In [3]:
from pathlib import Path
import shutil
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
#from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import datetime
import numpy as np

In [6]:
class TimeSeriesFeatures:
    def __init__(self, config: dict):
        self.len_history = config['len_history']  # количество строк в истории для создания окна

    def get_dummy_features(self, inp_df: pd.DataFrame, label_column_name: str):
        features, labels = [], []
        
        num_rows = inp_df.shape[0]

        # Применение скользящего окна
        for start in range(num_rows - self.len_history + 1):
            
            # Извлекаем окно из len_history строк
            window = inp_df.iloc[start:start + self.len_history]
            window_features = window.drop(columns=[label_column_name])
            
            # Разворачиваем параметры в один массив
            feature_vector = window_features.values.flatten()

            # Получаем метку из последней строки окна
            label = window[label_column_name].values[-1]

            features.append(feature_vector)
            labels.append(label)

        return features, labels


### Разделений файлов на тест и тренировку

In [8]:
source_directory = Path('/s/ls4/users/selishcheva/VKR/well_data')
train_directory = Path('/s/ls4/users/selishcheva/VKR/train_well_data')
test_directory = Path('/s/ls4/users/selishcheva/VKR/test_well_data')


for fp in source_directory.iterdir():
    if fp.is_file():
        well_number = fp.name.split('_')[1] 

        if well_number in ['01', '02', '03']:
            shutil.move(str(fp), str(train_directory / fp.name))
            
        elif well_number in ['04', '05', '06']:
            shutil.move(str(fp), str(test_directory / fp.name))

print("Файлы успешно распределены по выборкам.")

Файлы успешно распределены по выборкам.


### Тренировочная выборка

In [4]:
# Проверка размерности, в одной из таблиц для тренировочной выборки было повторение колонок "Вес на крюке" и "Давление"
# Я просто удалила повтор
train_directory = Path('/s/ls4/users/selishcheva/VKR/train_well_data')

for fp in train_directory.iterdir():
    if fp.is_file():
        
        df_i = pd.read_csv(fp, index_col=0, sep=',')
        #print (df_i.shape[1])
    

In [ ]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))


config_1 = {'len_history': 5}
ts_feats = TimeSeriesFeatures(config=config_1)

train_features, train_labels = [], []

for fp in train_directory.iterdir():
    
    if fp.is_file():
        
        df_i = pd.read_csv(fp, index_col=0, sep=',')

        features_df, labels_df = ts_feats.get_dummy_features(inp_df=df_i, label_column_name='Операции (euc)')
        
        train_features.extend(features_df)
        train_labels.extend(labels_df)

# Преобразуем в массивы numpy (если нужно)
train_features = np.array(train_features)
train_labels = np.array(train_labels)



# Место для сохранения all_features и all_labels
# Например, можно использовать pickle или сохранение в CSV
pd.DataFrame(train_features).to_csv('train_features.csv', index=False)
pd.DataFrame(train_labels).to_csv('train_labels.csv', index=False)



finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))

Время старта: 2024-10-22 16:08:40.794745


In [3]:
tr_f = pd.read_csv('/s/ls4/users/selishcheva/VKR/train_features.csv')
tr_f.shape

(7834840, 65)

In [4]:
tr_l = pd.read_csv('/s/ls4/users/selishcheva/VKR/train_labels.csv')
tr_l.shape

(7834840, 1)

### Тестовая выборка

In [9]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))


config_1 = {'len_history': 5}
ts_feats = TimeSeriesFeatures(config=config_1)

# Списки для хранения признаков и меток тестовой выборки
test_features, test_labels = [], []

for fp in test_directory.iterdir():
    
    if fp.is_file():
        
        df_i = pd.read_csv(fp, index_col=0, sep=',')
        
        features_df, labels_df = ts_feats.get_dummy_features(inp_df=df_i, label_column_name='Операции (euc)')
        
        test_features.extend(features_df)
        test_labels.extend(labels_df)

# Преобразуем в массивы numpy (если нужно)
test_features = np.array(test_features)  
test_labels = np.array(test_labels)      

# Место для сохранения test_features и test_labels
pd.DataFrame(test_features).to_csv('test_features.csv', index=False)
pd.DataFrame(test_labels).to_csv('test_labels.csv', index=False)



finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))

Время старта: 2024-10-23 12:23:41.670949
Время окончания: 2024-10-23 12:44:56.068505
Время работы: 0:21:14.397556


In [10]:
test_f = pd.read_csv('/s/ls4/users/selishcheva/VKR/test_features.csv')
test_f.shape

(3447639, 65)

In [11]:
test_l = pd.read_csv('/s/ls4/users/selishcheva/VKR/test_labels.csv')
test_l.shape

(3447639, 1)

### CatBoosting

In [1]:
from catboost import CatBoostClassifier
from pathlib import Path
import shutil
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score
import datetime
import numpy as np

def normalize_data(df, method):
    """
    Нормализует данные в DataFrame по указанному методу.

    Args:
        df (pd.DataFrame): Входной DataFrame, содержащий данные, которые необходимо нормализовать.
        method (str): Метод нормализации ('min_max', 'z_score', 'robust').
    """
    if method == 'min_max':
        scaler = MinMaxScaler()
    elif method == 'z_score':
        scaler = StandardScaler()
    elif method == 'robust':
        scaler = RobustScaler()
    else:
        raise ValueError("Invalid normalization method")

    df[df.columns.difference(['Операции (euc)'])] = scaler.fit_transform(df[df.columns.difference(['Операции (euc)'])])

def train_model_cat_boost(tr_f, tr_l, test_f, test_l):
    
    """
    Обучает модель CatBoost на данных из DataFrame и выводит отчет о классификации.

    Args:
        df (pd.DataFrame): Входной DataFrame, содержащий данные и целевую переменную 'Операции (euc)'.
    """

    # Разделение на обучающую и тестовую выборки
    
    X_train = pd.read_csv(tr_f)
    y_train = pd.read_csv(tr_l)  
    X_test = pd.read_csv(test_f)
    y_test = pd.read_csv(test_l)

    model = CatBoostClassifier(iterations=300, learning_rate=0.05, depth=9)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Подсчет accuracy и F1-score
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Вывод отчета о классификации
    print(classification_report(y_test, y_pred))
    print(f'Accuracy: {accuracy:.4f}')
    print(f'F1 Score: {f1:.4f}')


In [2]:
start = datetime.datetime.now()
print('Время старта: ' + str(start))

train_model_cat_boost('train_features.csv', 'train_labels.csv', 'test_features.csv', 'test_labels.csv')

finish = datetime.datetime.now()
print('Время окончания: ' + str(finish))
print('Время работы: ' + str(finish - start))

Время старта: 2024-10-27 18:17:14.027775
0:	learn: 2.3744251	total: 16.2s	remaining: 1h 20m 58s
1:	learn: 2.1246173	total: 32.3s	remaining: 1h 20m 13s
2:	learn: 1.9448307	total: 48.3s	remaining: 1h 19m 37s
3:	learn: 1.8070111	total: 1m 4s	remaining: 1h 18m 56s
4:	learn: 1.6931017	total: 1m 20s	remaining: 1h 19m 16s
5:	learn: 1.5984066	total: 1m 36s	remaining: 1h 19m 4s
6:	learn: 1.5165792	total: 1m 53s	remaining: 1h 19m
7:	learn: 1.4458699	total: 2m 10s	remaining: 1h 19m 39s
8:	learn: 1.3837016	total: 2m 29s	remaining: 1h 20m 29s
9:	learn: 1.3275367	total: 2m 47s	remaining: 1h 20m 46s
10:	learn: 1.2750173	total: 3m 4s	remaining: 1h 20m 56s
11:	learn: 1.2280687	total: 3m 22s	remaining: 1h 21m 4s
12:	learn: 1.1855364	total: 3m 41s	remaining: 1h 21m 29s
13:	learn: 1.1466698	total: 3m 59s	remaining: 1h 21m 39s
14:	learn: 1.1088767	total: 4m 18s	remaining: 1h 21m 44s
15:	learn: 1.0755991	total: 4m 36s	remaining: 1h 21m 43s
16:	learn: 1.0445296	total: 4m 54s	remaining: 1h 21m 40s
17:	learn: 

/s/ls4/users/selishcheva/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/s/ls4/users/selishcheva/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

         0.0       0.04      0.07      0.05     75960
         1.0       0.93      0.99      0.96    408183
         2.0       0.96      0.94      0.95    198959
         3.0       0.66      0.08      0.14     19444
         4.0       0.75      0.32      0.45    111315
         5.0       0.76      0.55      0.63     50173
         6.0       0.68      0.08      0.14     65634
         7.0       0.82      0.25      0.38    344466
         8.0       0.00      0.00      0.00      1501
         9.0       0.90      0.69      0.78    187886
        10.0       0.00      0.00      0.00       399
        11.0       0.53      0.76      0.62    198566
        12.0       0.34      0.50      0.41     83317
        13.0       0.82      0.87      0.84   1155210
        14.0       0.32      0.63      0.42    289446
        15.0       0.49      0.04      0.07      8413
        16.0       0.42      0.27      0.33    248707
        17.0       0.00    

/s/ls4/users/selishcheva/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
